In [22]:
import pandas as pd
from datetime import datetime
from calendar import monthrange

df_18 = pd.read_csv('../MIGTABO20X.csv',sep="|")
df_columns_18 = pd.read_csv('Campos MIGTAB020.csv',sep=";")
df_18.drop('Unnamed: 16',axis=1,inplace= True)
df_18.columns = df_columns_18['NombreColumna'].tolist()

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

In [23]:

# Asignar tu DataFrame
df = df_18
errores = []

# Validación: di_operacion NO NULL y ≠ 0
df["di_operacion"] = pd.to_numeric(df["di_operacion"], errors="coerce")
val = df["di_operacion"].isnull() | (df["di_operacion"] == 0)
errores.append(df[val][["di_operacion"]].assign(campo_error="di_operacion"))

# Validación: di_dividendo NO NULL
val = df["di_dividendo"].isnull()
errores.append(df[val][["di_operacion"]].assign(campo_error="di_dividendo"))

# Validación extra: cada di_operacion debe tener al menos un di_dividendo == 1
operaciones_con_dividendo_1 = df.groupby("di_operacion")["di_dividendo"].min() == 1
operaciones_invalidas = operaciones_con_dividendo_1[~operaciones_con_dividendo_1].index
errores_dividendo = df[df["di_operacion"].isin(operaciones_invalidas)][["di_operacion"]].drop_duplicates()
errores_dividendo = errores_dividendo.assign(campo_error="di_dividendo (falta cuota 1)")
errores.append(errores_dividendo)

# Validación: di_fecha_ini NO NULL
val = df["di_fecha_ini"].isnull()
errores.append(df[val][["di_operacion"]].assign(campo_error="di_fecha_ini"))

# Validación: di_fecha_ven NO NULL
val = df["di_fecha_ven"].isnull()
errores.append(df[val][["di_operacion"]].assign(campo_error="di_fecha_ven"))

# Validación: di_de_capital ∈ ["S", "N"]
val = ~df["di_de_capital"].isin(["S", "N"])
errores.append(df[val][["di_operacion"]].assign(campo_error="di_de_capital"))

# Validación: di_de_interes ∈ ["S", "N"]
val = ~df["di_de_interes"].isin(["S", "N"])
errores.append(df[val][["di_operacion"]].assign(campo_error="di_de_interes"))

# Validación: di_gracia NO NULL y numérico
val = df["di_gracia"].isnull() | ~df["di_gracia"].astype(str).str.isnumeric()
errores.append(df[val][["di_operacion"]].assign(campo_error="di_gracia"))

# Validación: di_gracia_disp NO NULL
val = df["di_gracia_disp"].isnull()
errores.append(df[val][["di_operacion"]].assign(campo_error="di_gracia_disp"))

# Validación: di_estado ∈ [0, 1, 2, 3]
val = df["di_estado"].isnull() | ~df["di_estado"].isin([0, 1, 2, 3])
errores.append(df[val][["di_operacion"]].assign(campo_error="di_estado"))

# Validación: di_dias_cuota NO NULL
val = df["di_dias_cuota"].isnull()
errores.append(df[val][["di_operacion"]].assign(campo_error="di_dias_cuota"))

# Validación: di_intento == 0
val = df["di_intento"].isnull() | (df["di_intento"] != 0)
errores.append(df[val][["di_operacion"]].assign(campo_error="di_intento"))

# Validación: di_prorroga ∈ ["S", "N"]
val = df["di_prorroga"].isnull() | ~df["di_prorroga"].isin(["S", "N"])
errores.append(df[val][["di_operacion"]].assign(campo_error="di_prorroga"))

# Validación: di_subs_cuota NO NULL
val = df["di_subs_cuota"].isnull()
errores.append(df[val][["di_operacion"]].assign(campo_error="di_subs_cuota"))

# Validación: di_subs_acumulado NO NULL
val = df["di_subs_acumulado"].isnull()
errores.append(df[val][["di_operacion"]].assign(campo_error="di_subs_acumulado"))

# Concatenar todos los errores
df_errores = pd.concat(errores, ignore_index=True)

# Mostrar errores si existen
if df_errores.empty:
    print("✅ Todos los datos son válidos")
else:
    print("🔴 Se encontraron errores:")
    print(df_errores)




🔴 Se encontraron errores:
   di_operacion                   campo_error
0          5001  di_dividendo (falta cuota 1)


In [28]:
df_20 = df[~df["di_operacion"].isin(df_errores["di_operacion"])]

fecha_formateada = pd.Timestamp.now().strftime("%Y%m%d%H%M%S")